In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gurobipy as gb
import os
from pickle import dump, load
from time import process_time
from source import*

path = os.getcwd()

In [2]:
file = open(path+"/Results/Configurations/Open Stations/open_stations_18","rb"); S = load(file); file.close()

K_s = {}; K ={}
for sc in range(25):
    file = open(path+f"/Data/K/K_sc{sc}", "rb")
    K[sc] = load(file); file.close()

    file = open(path+f"/Data/K_s10/Ks_sc{sc}", "rb")
    K_s[sc] = load(file); file.close()

stress = {}
for s in S:
    stress[s] = np.mean(np.array([len(K_s[sc][s]) for sc in range(25)]))

max_s = max(stress, key=stress.get)
max_s, stress[max_s]

(459, 197.0)

In [3]:
def number_of_chargers(s,S,y,exhaust=False):

    routes = {}
    for sc in range(25):

        K, K_s, S_k, a, t = load_pickle(path+"/Data/",sc)
        S_k = {k:[s for s in S_k[k] if s in S] for k in K}
        a.update({("s",s):0 for s in S})
        a.update({("e",s):30 for s in S})
        t.update({("s",s):0 for s in S})
        t.update({("e",s):0 for s in S})

        mp = gb.Model("Restricted Master Problem")

        dummy_0 = {k:mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=1, name=f"st0_{k}") for k in K_s[s]}
        aux = mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=2, name=f"aux_{s}")

        vehic_assign = {}
        for k in K_s[s]:
            vehic_assign[k] = mp.addConstr(dummy_0[k] == 1, f"V{k}_assignment")

        st_conv = mp.addConstr(aux <= y, f"S{s}_convexity")

        mp.setParam("OutputFlag",0)

        time0 = process_time(); i = 0
        lbd = []

        while True:
            
            mp.update()
            mp.optimize()

            pi = {k:vehic_assign[k].getAttr("Pi") for k in K_s[s]}
            sigma = st_conv.getAttr("Pi")

            print(f"\t\tIteration {i}:\t\tMP obj: {round(mp.getObjective().getValue(),2)}\ttime: {round(process_time()-time0,2)}s")
            i += 1

            V,A,rc = get_graph(s,K_s[s],a,pi,sigma)
            ext = vertices_extensions(V,A)

            routes_DFS = []; covered_nodes = set()
            if exhaust: label_DFS_exhaustive(v="s",rP=0,tP=0,qP=0,P=[],cK=covered_nodes,L=routes_DFS,s=s,r=rc,t=t,a=a,ext=ext)
            else: label_DFS(v="s",rP=0,tP=0,qP=0,P=[],cK=covered_nodes,L=routes_DFS,s=s,r=rc,t=t,a=a,ext=ext)
            opt = len(routes_DFS)

            for l in routes_DFS:
                col = {k:1 if k in l else 0 for k in K_s[s]}
                new_Col = gb.Column(col.values(),vehic_assign.values())
                lbd.append(mp.addVar(vtype=gb.GRB.CONTINUOUS,obj=0,ub=1,column=new_Col))

                mp.chgCoeff(st_conv,lbd[-1],1)

            if opt == 0: mpsol = mp.getObjective().getValue(); break
        
        for v in mp.getVars():
            v.vtype = gb.GRB.BINARY
        if mpsol > 0: mp.setParam("MIPGap",0.01)
        mp.setParam("MIPFocus",2)
        mp.update(); mp.optimize()

        print(f"IMP solution: {mp.getObjective().getValue()} infeasible vehicles")
        routes[sc] = []
        for l in lbd:
            if l.X > 0.5:
                col = [k for k in K_s[s] if mp.getCoeff(vehic_assign[k],l) == 1]
                routes[sc].append(col)
        
        break

    return routes

In [4]:
routes_DFS = number_of_chargers(max_s,S,8)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
		Iteration 0:		MP obj: 192.0	time: 0.0s
		Iteration 1:		MP obj: 51.0	time: 2.22s
		Iteration 2:		MP obj: 51.0	time: 4.33s
		Iteration 3:		MP obj: 51.0	time: 6.34s
		Iteration 4:		MP obj: 51.0	time: 8.0s
		Iteration 5:		MP obj: 51.0	time: 9.77s
		Iteration 6:		MP obj: 51.0	time: 11.19s
		Iteration 7:		MP obj: 51.0	time: 12.62s
		Iteration 8:		MP obj: 51.0	time: 14.27s
		Iteration 9:		MP obj: 51.0	time: 15.94s
		Iteration 10:		MP obj: 51.0	time: 17.48s
		Iteration 11:		MP obj: 51.0	time: 19.12s
		Iteration 12:		MP obj: 51.0	time: 20.84s
		Iteration 13:		MP obj: 51.0	time: 22.67s
		Iteration 14:		MP obj: 50.99	time: 24.45s
		Iteration 15:		MP obj: 50.94	time: 26.11s
		Iteration 16:		MP obj: 50.65	time: 27.91s
		Iteration 17:		MP obj: 50.61	time: 29.66s
		Iteration 18:		MP obj: 50.59	time: 31.56s
		Iteration 19:		MP obj: 50.54	time: 33.89s
		Iteration 20:		MP obj: 50.51	time: 35.73s
		Iteration 21:	

In [5]:
print(f"DFS Labeling covered vehicles: {sum(len(r) for r in routes_DFS[0])}")

DFS Labeling covered vehicles: 141


In [4]:
routes_DFS_exhaustive = number_of_chargers(max_s,S,8,exhaust=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
		Iteration 0:		MP obj: 192.0	time: 0.0s


TypeError: label_DFS_exhaustive() missing 1 required positional argument: 'ext'

In [ ]:
print(f"DFS Labeling covered vehicles: {sum(len(r) for r in routes_DFS)}")
print(f"Exhaustive DFS Labeling covered vehicles: {sum(len(r) for r in routes_DFS_exhaustive)}")